In [1]:
# =============================================================================
# RUNTIME REGRESSION MODEL - Model Comparison
# =============================================================================
# Goal: Predict forward_runtime given circuit features + threshold
# Dataset: training_data.csv (0.99 fidelity threshold, CPU-only)
# Key: min_threshold is used as a FEATURE (not target)
# Primary metric: MAPE (lower is better)
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor,
    AdaBoostRegressor, BaggingRegressor
)
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

# -----------------------------------------------------------------------------
# 1. LOAD DATA
# -----------------------------------------------------------------------------
df = pd.read_csv("training_data.csv")

print("=" * 70)
print("RUNTIME REGRESSION MODEL")
print("=" * 70)
print()
print(f"Dataset: training_data.csv")
print(f"Total samples: {len(df)}")
print(f"Unique files: {df['file'].nunique()}")
print()

# Target is forward_runtime
print("Target: forward_runtime")
print(f"  Min: {df['forward_runtime'].min():.2f} seconds")
print(f"  Max: {df['forward_runtime'].max():.2f} seconds")
print(f"  Mean: {df['forward_runtime'].mean():.2f} seconds")
print(f"  Median: {df['forward_runtime'].median():.2f} seconds")
print(f"  Std: {df['forward_runtime'].std():.2f} seconds")
print()

print("Runtime distribution:")
for p in [25, 50, 75, 90, 95, 99]:
    val = np.percentile(df['forward_runtime'], p)
    print(f"  {p}th percentile: {val:.2f} seconds")
print()

# -----------------------------------------------------------------------------
# 2. FEATURE ENGINEERING
# -----------------------------------------------------------------------------
def engineer_features(df):
    """Create domain-specific features for runtime prediction."""
    X = df.copy()

    # Interaction features
    X['degree_x_qubits'] = X['avg_qubit_degree'] * X['n_qubits']
    X['degree_x_depth'] = X['avg_qubit_degree'] * X['crude_depth']
    X['entanglement_complexity'] = X['n_unique_edges'] * X['avg_qubit_degree']
    X['entanglement_per_qubit'] = X['n_unique_edges'] / (X['n_qubits'] + 1)

    # Ratio features
    X['cx_ratio'] = X['n_cx'] / (X['n_total_gates'] + 1)
    X['multi_qubit_ratio'] = (X['n_2q_gates'] + X['n_3q_gates']) / (X['n_total_gates'] + 1)
    X['gates_per_depth'] = X['n_total_gates'] / (X['crude_depth'] + 1)
    X['depth_per_qubit'] = X['crude_depth'] / (X['n_qubits'] + 1)
    X['edge_density'] = X['n_unique_edges'] / (X['n_qubits'] * (X['n_qubits'] - 1) / 2 + 1)
    X['edge_repetition_ratio'] = X['n_edge_repetitions'] / (X['n_unique_edges'] + 1)

    # Log features (helps with skewed distributions)
    X['log_qubits'] = np.log1p(X['n_qubits'])
    X['log_depth'] = np.log1p(X['crude_depth'])
    X['log_gates'] = np.log1p(X['n_total_gates'])
    X['log_threshold'] = np.log2(X['min_threshold'] + 1)

    # Complexity scores
    X['complexity_score'] = X['n_qubits'] * X['crude_depth'] * X['avg_qubit_degree'] / 1000
    X['sim_difficulty'] = X['n_qubits'] ** 1.5 * X['entanglement_pressure']

    # Threshold-based features
    X['threshold_x_qubits'] = X['min_threshold'] * X['n_qubits']
    X['threshold_x_gates'] = X['min_threshold'] * X['n_total_gates']

    return X

# Apply feature engineering
X_eng = engineer_features(df)

# Target variable - use LOG transform for skewed runtime
y = df['forward_runtime'].values
y_log = np.log1p(y)

# Groups for cross-validation
groups = df['file'].values

# Columns to drop (non-features)
drop_cols = ["forward_runtime", "file",
             "max_fidelity_achieved", "n_thresholds_tested", "threshold_runtime"]
drop_cols = [c for c in drop_cols if c in X_eng.columns]
X_eng = X_eng.drop(columns=drop_cols)

print(f"min_threshold included as feature: {'min_threshold' in X_eng.columns}")

# One-hot encode categorical columns
cat_cols = X_eng.select_dtypes(exclude=[np.number]).columns.tolist()
print(f"Categorical columns: {cat_cols}")
X_eng = pd.get_dummies(X_eng, columns=cat_cols)

# Prepare arrays
X = X_eng.values.astype(np.float64)
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

print(f"Feature matrix shape: {X.shape}")
print()

# -----------------------------------------------------------------------------
# 3. MODEL COMPARISON
# -----------------------------------------------------------------------------
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

print(f"Using {n_splits}-fold GroupKFold (grouped by file)")
print()

models = {
    'XGBoost': XGBRegressor(
        n_estimators=500, max_depth=6, learning_rate=0.05,
        random_state=42, verbosity=0
    ),
    'LightGBM': LGBMRegressor(
        n_estimators=500, max_depth=6, learning_rate=0.05,
        random_state=42, verbose=-1
    ),
    'RandomForest': RandomForestRegressor(
        n_estimators=500, max_depth=15, min_samples_leaf=2,
        random_state=42, n_jobs=-1
    ),
    'ExtraTrees': ExtraTreesRegressor(
        n_estimators=500, max_depth=15, min_samples_leaf=2,
        random_state=42, n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=500, max_depth=5, learning_rate=0.05,
        random_state=42
    ),
    'AdaBoost': AdaBoostRegressor(
        n_estimators=200, learning_rate=0.05, random_state=42
    ),
    'SVR_linear': SVR(kernel='linear', C=1.0),
}

print("=" * 70)
print("MODEL EVALUATION (predicting log(runtime), then transforming back)")
print("=" * 70)
print()

results = []

for name, model in models.items():
    print(f"Evaluating {name}...")

    y_pred_all = np.zeros(len(y))

    for fold_idx, (train_idx, test_idx) in enumerate(gkf.split(X, y_log, groups)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_idx])
        X_test = scaler.transform(X[test_idx])

        X_train = np.clip(X_train, -10, 10)
        X_test = np.clip(X_test, -10, 10)

        model_fold = model.__class__(**model.get_params())
        model_fold.fit(X_train, y_log[train_idx])

        y_pred_log = model_fold.predict(X_test)
        y_pred = np.expm1(y_pred_log)
        y_pred = np.maximum(y_pred, 0)

        y_pred_all[test_idx] = y_pred

    rmse = np.sqrt(mean_squared_error(y, y_pred_all))
    mae = mean_absolute_error(y, y_pred_all)
    r2 = r2_score(y, y_pred_all)
    mape = np.mean(np.abs(y - y_pred_all) / np.maximum(y, 1.0)) * 100
    medape = np.median(np.abs(y - y_pred_all) / np.maximum(y, 1.0)) * 100

    results.append({
        'model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape,
        'MedAPE': medape
    })

print()
print("=" * 70)
print("RESULTS - Sorted by MAPE (lower is better)")
print("=" * 70)
print()
print(f"{'Model':<20} {'MAPE%':>8} {'MedAPE%':>8} {'MAE':>10} {'RMSE':>10} {'R²':>8}")
print("-" * 70)

for r in sorted(results, key=lambda x: x['MAPE']):
    print(f"{r['model']:<20} {r['MAPE']:>8.1f} {r['MedAPE']:>8.1f} {r['MAE']:>10.2f} "
          f"{r['RMSE']:>10.2f} {r['R2']:>8.4f}")

print()
best = min(results, key=lambda x: x['MAPE'])
print(f"Best Model: {best['model']}")
print(f"  MAPE: {best['MAPE']:.1f}%")
print(f"  MedAPE: {best['MedAPE']:.1f}%")
print(f"  MAE: {best['MAE']:.2f} seconds")
print(f"  R²: {best['R2']:.4f}")

RUNTIME REGRESSION MODEL

Dataset: training_data.csv
Total samples: 1107
Unique files: 576

Target: forward_runtime
  Min: 3192.51 seconds
  Max: 9158881.81 seconds
  Mean: 286512.97 seconds
  Median: 74048.02 seconds
  Std: 667834.21 seconds

Runtime distribution:
  25th percentile: 25379.24 seconds
  50th percentile: 74048.02 seconds
  75th percentile: 243357.39 seconds
  90th percentile: 740783.87 seconds
  95th percentile: 1331608.45 seconds
  99th percentile: 2752661.68 seconds

min_threshold included as feature: True
Categorical columns: ['backend', 'precision']
Feature matrix shape: (1107, 84)

Using 5-fold GroupKFold (grouped by file)

MODEL EVALUATION (predicting log(runtime), then transforming back)

Evaluating XGBoost...
Evaluating LightGBM...
Evaluating RandomForest...
Evaluating ExtraTrees...
Evaluating GradientBoosting...
Evaluating AdaBoost...
Evaluating SVR_linear...

RESULTS - Sorted by MAPE (lower is better)

Model                   MAPE%  MedAPE%        MAE       RMS

In [2]:
# =============================================================================
# FEATURE SELECTION - Find optimal feature count
# =============================================================================

from sklearn.ensemble import RandomForestRegressor

print("=" * 70)
print("FEATURE SELECTION")
print("=" * 70)
print()

# Get feature importance using RandomForest
print("Training RandomForest to get feature importances...")
scaler_init = StandardScaler()
X_scaled_init = scaler_init.fit_transform(X)

rf_imp = RandomForestRegressor(
    n_estimators=500, max_depth=15, min_samples_leaf=2,
    random_state=42, n_jobs=-1
)
rf_imp.fit(X_scaled_init, y_log)

importance_df = pd.DataFrame({
    'feature': X_eng.columns.tolist(),
    'importance': rf_imp.feature_importances_
}).sort_values('importance', ascending=False)

print()
print("Top 20 most important features:")
print("-" * 50)
for _, row in importance_df.head(20).iterrows():
    print(f"  {row['feature']:<35} {row['importance']:.4f}")
print()

# -----------------------------------------------------------------------------
# Try different feature counts with the best model from cell 1
# -----------------------------------------------------------------------------
print("Optimal number of features (using best model from above):")
print("-" * 50)

best_model_name = best['model']
best_model_template = models[best_model_name]

feature_sweep_results = []

for top_k in list(range(10, len(importance_df), 10)) + [len(importance_df)]:
    if top_k > len(importance_df):
        continue

    top_features = importance_df.head(top_k)['feature'].tolist()
    X_top = X_eng[top_features].values.astype(np.float64)
    X_top = np.nan_to_num(X_top, nan=0.0, posinf=0.0, neginf=0.0)

    y_pred_all = np.zeros(len(y))

    for train_idx, test_idx in gkf.split(X_top, y_log, groups):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_top[train_idx])
        X_test = scaler.transform(X_top[test_idx])
        X_train = np.clip(X_train, -10, 10)
        X_test = np.clip(X_test, -10, 10)

        model_fold = best_model_template.__class__(**best_model_template.get_params())
        model_fold.fit(X_train, y_log[train_idx])
        y_pred_log = model_fold.predict(X_test)
        y_pred_all[test_idx] = np.maximum(np.expm1(y_pred_log), 0)

    mape = np.mean(np.abs(y - y_pred_all) / np.maximum(y, 1.0)) * 100
    mae = mean_absolute_error(y, y_pred_all)
    feature_sweep_results.append({'k': top_k, 'MAPE': mape, 'MAE': mae})
    print(f"  Top {top_k:2d} features: MAPE = {mape:.1f}%, MAE = {mae:.2f}s")

# Pick best feature count (lowest MAPE)
best_k_result = min(feature_sweep_results, key=lambda x: x['MAPE'])
BEST_K = best_k_result['k']
TOP_FEATURES = importance_df.head(BEST_K)['feature'].tolist()

print()
print(f"Best feature count: {BEST_K} (MAPE = {best_k_result['MAPE']:.1f}%)")
print()
print(f"Selected Top {BEST_K} Features:")
for i, feat in enumerate(TOP_FEATURES, 1):
    imp = importance_df[importance_df['feature'] == feat]['importance'].values[0]
    print(f"  {i:2d}. {feat:<35} ({imp:.4f})")
print()

# Also re-compare all models with the selected features
print("=" * 70)
print(f"MODEL RE-COMPARISON WITH TOP {BEST_K} FEATURES")
print("=" * 70)
print()

X_top = X_eng[TOP_FEATURES].values.astype(np.float64)
X_top = np.nan_to_num(X_top, nan=0.0, posinf=0.0, neginf=0.0)

results_top = []

for name, model in models.items():
    print(f"Evaluating {name}...")
    y_pred_all = np.zeros(len(y))

    for train_idx, test_idx in gkf.split(X_top, y_log, groups):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_top[train_idx])
        X_test = scaler.transform(X_top[test_idx])
        X_train = np.clip(X_train, -10, 10)
        X_test = np.clip(X_test, -10, 10)

        model_fold = model.__class__(**model.get_params())
        model_fold.fit(X_train, y_log[train_idx])
        y_pred_log = model_fold.predict(X_test)
        y_pred_all[test_idx] = np.maximum(np.expm1(y_pred_log), 0)

    rmse = np.sqrt(mean_squared_error(y, y_pred_all))
    mae = mean_absolute_error(y, y_pred_all)
    r2 = r2_score(y, y_pred_all)
    mape = np.mean(np.abs(y - y_pred_all) / np.maximum(y, 1.0)) * 100
    medape = np.median(np.abs(y - y_pred_all) / np.maximum(y, 1.0)) * 100

    results_top.append({
        'model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape,
        'MedAPE': medape
    })

print()
print(f"{'Model':<20} {'MAPE%':>8} {'MedAPE%':>8} {'MAE':>10} {'RMSE':>10} {'R²':>8}")
print("-" * 70)

for r in sorted(results_top, key=lambda x: x['MAPE']):
    print(f"{r['model']:<20} {r['MAPE']:>8.1f} {r['MedAPE']:>8.1f} {r['MAE']:>10.2f} "
          f"{r['RMSE']:>10.2f} {r['R2']:>8.4f}")

best_top = min(results_top, key=lambda x: x['MAPE'])
BEST_MODEL_NAME = best_top['model']
print()
print(f"Best model with top {BEST_K} features: {BEST_MODEL_NAME} (MAPE = {best_top['MAPE']:.1f}%)")

FEATURE SELECTION

Training RandomForest to get feature importances...

Top 20 most important features:
--------------------------------------------------
  max_gate_span                       0.5690
  log_qubits                          0.0699
  n_qubits                            0.0631
  n_measure                           0.0463
  circuit_density                     0.0367
  precision_single                    0.0340
  precision_double                    0.0295
  threshold_x_qubits                  0.0218
  n_u3                                0.0205
  n_u                                 0.0193
  degree_x_qubits                     0.0099
  n_1q_gates                          0.0098
  n_h                                 0.0064
  avg_qubit_degree                    0.0042
  1q_gates_per_qubit                  0.0037
  cx_ratio                            0.0035
  n_2q_gates                          0.0034
  std_gate_span                       0.0031
  gates_per_layer_estimate         

In [3]:
# =============================================================================
# HYPERPARAMETER TUNING - Optuna on best model (minimizing MAPE)
# =============================================================================

import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("=" * 70)
print(f"HYPERPARAMETER TUNING: {BEST_MODEL_NAME} with Top {BEST_K} Features")
print("=" * 70)
print()

# Prepare feature matrix
X_top = X_eng[TOP_FEATURES].values.astype(np.float64)
X_top = np.nan_to_num(X_top, nan=0.0, posinf=0.0, neginf=0.0)

# Define objective functions for each possible best model
def make_objective(model_name):
    def objective(trial):
        if model_name == 'XGBoost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, 20),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
                'gamma': trial.suggest_float('gamma', 1e-8, 5.0, log=True),
                'random_state': 42,
                'verbosity': 0
            }
            model_class = XGBRegressor
        elif model_name == 'LightGBM':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, 20),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 10, 200),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
                'random_state': 42,
                'verbose': -1
            }
            model_class = LGBMRegressor
        elif model_name == 'GradientBoosting':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5, log=True),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 15),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'random_state': 42
            }
            model_class = GradientBoostingRegressor
        elif model_name == 'RandomForest':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 5, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'random_state': 42,
                'n_jobs': -1
            }
            model_class = RandomForestRegressor
        elif model_name == 'ExtraTrees':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 5, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'random_state': 42,
                'n_jobs': -1
            }
            model_class = ExtraTreesRegressor
        else:
            raise ValueError(f"No tuning defined for {model_name}")

        y_pred_all = np.zeros(len(y))

        for train_idx, test_idx in gkf.split(X_top, y_log, groups):
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_top[train_idx])
            X_test = scaler.transform(X_top[test_idx])
            X_train = np.clip(X_train, -10, 10)
            X_test = np.clip(X_test, -10, 10)

            model = model_class(**params)
            model.fit(X_train, y_log[train_idx])
            y_pred_log = model.predict(X_test)
            y_pred_all[test_idx] = np.maximum(np.expm1(y_pred_log), 0)

        mape = np.mean(np.abs(y - y_pred_all) / np.maximum(y, 1.0)) * 100
        r2 = r2_score(y, y_pred_all)
        mae = mean_absolute_error(y, y_pred_all)
        trial.set_user_attr('r2', r2)
        trial.set_user_attr('mae', mae)
        return mape

    return objective

# Run Optuna (minimize MAPE)
N_TRIALS = 200
print(f"Running {N_TRIALS} Optuna trials for {BEST_MODEL_NAME}...")
print(f"Objective: minimize MAPE")
print()

sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(make_objective(BEST_MODEL_NAME), n_trials=N_TRIALS, show_progress_bar=True)

print()
print("=" * 70)
print("TUNING RESULTS")
print("=" * 70)
print()
print(f"Best MAPE: {study.best_value:.1f}%")
print(f"Best R²:   {study.best_trial.user_attrs['r2']:.4f}")
print(f"Best MAE:  {study.best_trial.user_attrs['mae']:.2f}s")
print()
print("Best Hyperparameters:")
BEST_PARAMS = study.best_params.copy()
for k, v in BEST_PARAMS.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.6f}")
    else:
        print(f"  {k}: {v}")
print()

# Final evaluation with best params
print("=" * 70)
print("FINAL CROSS-VALIDATED METRICS")
print("=" * 70)
print()

# Add fixed params back
if BEST_MODEL_NAME == 'XGBoost':
    BEST_PARAMS['random_state'] = 42
    BEST_PARAMS['verbosity'] = 0
    best_model_class = XGBRegressor
elif BEST_MODEL_NAME == 'LightGBM':
    BEST_PARAMS['random_state'] = 42
    BEST_PARAMS['verbose'] = -1
    best_model_class = LGBMRegressor
elif BEST_MODEL_NAME == 'GradientBoosting':
    BEST_PARAMS['random_state'] = 42
    best_model_class = GradientBoostingRegressor
elif BEST_MODEL_NAME == 'RandomForest':
    BEST_PARAMS['random_state'] = 42
    BEST_PARAMS['n_jobs'] = -1
    best_model_class = RandomForestRegressor
elif BEST_MODEL_NAME == 'ExtraTrees':
    BEST_PARAMS['random_state'] = 42
    BEST_PARAMS['n_jobs'] = -1
    best_model_class = ExtraTreesRegressor

y_pred_final = np.zeros(len(y))

for train_idx, test_idx in gkf.split(X_top, y_log, groups):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_top[train_idx])
    X_test = scaler.transform(X_top[test_idx])
    X_train = np.clip(X_train, -10, 10)
    X_test = np.clip(X_test, -10, 10)

    model = best_model_class(**BEST_PARAMS)
    model.fit(X_train, y_log[train_idx])
    y_pred_final[test_idx] = np.maximum(np.expm1(model.predict(X_test)), 0)

final_mape = np.mean(np.abs(y - y_pred_final) / np.maximum(y, 1.0)) * 100
final_medape = np.median(np.abs(y - y_pred_final) / np.maximum(y, 1.0)) * 100

print(f"  MAPE:   {final_mape:.1f}%")
print(f"  MedAPE: {final_medape:.1f}%")
print(f"  MAE:    {mean_absolute_error(y, y_pred_final):.2f}s")
print(f"  RMSE:   {np.sqrt(mean_squared_error(y, y_pred_final)):.2f}s")
print(f"  R²:     {r2_score(y, y_pred_final):.4f}")
print()
print(f"Baseline (all features, default params): MAPE = {best['MAPE']:.1f}%")
print(f"After feature selection + tuning:         MAPE = {final_mape:.1f}%")

HYPERPARAMETER TUNING: GradientBoosting with Top 50 Features

Running 200 Optuna trials for GradientBoosting...
Objective: minimize MAPE



Best trial: 179. Best value: 9.55613: 100%|██████████| 200/200 [53:58<00:00, 16.19s/it]



TUNING RESULTS

Best MAPE: 9.6%
Best R²:   0.9666
Best MAE:  31260.56s

Best Hyperparameters:
  n_estimators: 794
  max_depth: 5
  learning_rate: 0.011740
  min_samples_split: 3
  min_samples_leaf: 1
  subsample: 0.572111

FINAL CROSS-VALIDATED METRICS

  MAPE:   9.6%
  MedAPE: 6.9%
  MAE:    31260.56s
  RMSE:   121989.43s
  R²:     0.9666

Baseline (all features, default params): MAPE = 10.5%
After feature selection + tuning:         MAPE = 9.6%


In [6]:
# =============================================================================
# PRODUCTION RUNTIME PREDICTOR
# =============================================================================
# Trains on ALL data with tuned hyperparameters, saves to models/
# =============================================================================

import joblib
from comprehensive_features import QASMFeatureExtractor
from pathlib import Path

print("=" * 70)
print("PRODUCTION RUNTIME PREDICTOR")
print("=" * 70)
print()
print(f"Model: {BEST_MODEL_NAME}")
print(f"Features: Top {BEST_K}")
print()
print("Hyperparameters:")
for k, v in BEST_PARAMS.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.6f}")
    else:
        print(f"  {k}: {v}")
print()

# -------------------------------------------------------------------------
# CROSS-VALIDATED METRICS (before training on all data)
# -------------------------------------------------------------------------
print("=" * 70)
print("CROSS-VALIDATED METRICS (5-fold GroupKFold)")
print("=" * 70)
print()

y_pred_cv = np.zeros(len(y))
fold_metrics = []

for fold_idx, (train_idx, test_idx) in enumerate(gkf.split(X_top, y_log, groups)):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_top[train_idx])
    X_test = scaler.transform(X_top[test_idx])
    X_train = np.clip(X_train, -10, 10)
    X_test = np.clip(X_test, -10, 10)

    cv_model = best_model_class(**BEST_PARAMS)
    cv_model.fit(X_train, y_log[train_idx])
    y_pred_cv[test_idx] = np.maximum(np.expm1(cv_model.predict(X_test)), 0)

    # Per-fold metrics
    fold_y = y[test_idx]
    fold_pred = y_pred_cv[test_idx]
    fold_mape = np.mean(np.abs(fold_y - fold_pred) / np.maximum(fold_y, 1.0)) * 100
    fold_mae = mean_absolute_error(fold_y, fold_pred)
    fold_r2 = r2_score(fold_y, fold_pred)
    fold_metrics.append({'fold': fold_idx + 1, 'MAPE': fold_mape, 'MAE': fold_mae, 'R2': fold_r2})
    print(f"  Fold {fold_idx + 1}: MAPE = {fold_mape:.1f}%, MAE = {fold_mae:.2f}s, R² = {fold_r2:.4f}")

cv_mape = np.mean(np.abs(y - y_pred_cv) / np.maximum(y, 1.0)) * 100
cv_medape = np.median(np.abs(y - y_pred_cv) / np.maximum(y, 1.0)) * 100
cv_mae = mean_absolute_error(y, y_pred_cv)
cv_rmse = np.sqrt(mean_squared_error(y, y_pred_cv))
cv_r2 = r2_score(y, y_pred_cv)

print()
print(f"  Overall MAPE:   {cv_mape:.1f}%")
print(f"  Overall MedAPE: {cv_medape:.1f}%")
print(f"  Overall MAE:    {cv_mae:.2f}s")
print(f"  Overall RMSE:   {cv_rmse:.2f}s")
print(f"  Overall R²:     {cv_r2:.4f}")
print()

# -------------------------------------------------------------------------
# TRAIN ON ALL DATA
# -------------------------------------------------------------------------
prod_scaler = StandardScaler()
X_prod = prod_scaler.fit_transform(X_top)
X_prod = np.clip(X_prod, -10, 10)

prod_model = best_model_class(**BEST_PARAMS)
prod_model.fit(X_prod, y_log)
print(f"Production model trained on full dataset ({len(y_log)} samples).")
print()

# Save references for prediction
PRODUCTION_FEATURES = TOP_FEATURES
PRODUCTION_DROP_COLS = drop_cols

# -------------------------------------------------------------------------
# SAVE MODEL TO DISK
# -------------------------------------------------------------------------
model_artifact = {
    'model': prod_model,
    'scaler': prod_scaler,
    'features': PRODUCTION_FEATURES,
    'drop_cols': PRODUCTION_DROP_COLS,
    'model_name': BEST_MODEL_NAME,
    'best_params': BEST_PARAMS,
    'best_k': BEST_K,
    'cv_metrics': {
        'mape': cv_mape,
        'medape': cv_medape,
        'mae': cv_mae,
        'rmse': cv_rmse,
        'r2': cv_r2,
    },
}

save_path = Path("models/runtime_model.pkl")
joblib.dump(model_artifact, save_path)
print(f"Model saved to {save_path}")
print(f"  File size: {save_path.stat().st_size / 1024:.1f} KB")
print()

# -------------------------------------------------------------------------
# VERIFY LOAD WORKS
# -------------------------------------------------------------------------
loaded = joblib.load(save_path)
print("Verified model loads successfully.")
print(f"  Model type: {type(loaded['model']).__name__}")
print(f"  Features: {len(loaded['features'])}")
print()

# -------------------------------------------------------------------------
# PREDICTION FUNCTION
# -------------------------------------------------------------------------
def predict_runtime(file_path, precision, threshold):
    """
    Predict runtime in seconds for a circuit.

    Args:
        file_path: Path to the QASM file
        precision: 'single' or 'double'
        threshold: The min_threshold value to use

    Returns:
        float: Predicted runtime in seconds
    """
    features = QASMFeatureExtractor(file_path).extract_all()
    features['backend'] = 'CPU'
    features['precision'] = precision
    features['min_threshold'] = threshold

    X = engineer_features(pd.DataFrame([features]))

    # Drop non-feature cols
    for col in PRODUCTION_DROP_COLS:
        if col in X.columns:
            X = X.drop(columns=[col])

    # One-hot encode
    cat = X.select_dtypes(exclude=[np.number]).columns.tolist()
    X = pd.get_dummies(X, columns=cat)

    # Align with training features
    for col in PRODUCTION_FEATURES:
        if col not in X.columns:
            X[col] = 0

    X_final = X[PRODUCTION_FEATURES].values.astype(np.float64)
    X_final = np.nan_to_num(X_final, nan=0.0, posinf=0.0, neginf=0.0)
    X_final = np.clip(prod_scaler.transform(X_final), -10, 10)

    return float(np.expm1(prod_model.predict(X_final)[0]))

# Quick test on a few training files
print("=" * 70)
print("SANITY CHECK ON TRAINING FILES")
print("=" * 70)
print()

circuits_dir = Path("circuits_new")
test_rows = df.sample(n=min(10, len(df)), random_state=42)

print(f"{'File':<45} {'Prec':<7} {'Thresh':>6} {'True':>10} {'Pred':>10} {'Err%':>8}")
print("-" * 90)

for _, row in test_rows.iterrows():
    qasm_path = circuits_dir / row['file']
    pred = predict_runtime(qasm_path, row['precision'], int(row['min_threshold']))
    true = row['forward_runtime']
    err_pct = abs(pred - true) / max(true, 1) * 100
    print(f"{row['file']:<45} {row['precision']:<7} {int(row['min_threshold']):>6} "
          f"{true:>10.2f} {pred:>10.2f} {err_pct:>7.1f}%")

print()
print("=" * 70)
print("USAGE - Loading saved model")
print("=" * 70)
print()
print("  import joblib")
print("  artifact = joblib.load('models/runtime_model.pkl')")
print("  model = artifact['model']")
print("  scaler = artifact['scaler']")
print("  features = artifact['features']")
print("  cv = artifact['cv_metrics']  # MAPE, MedAPE, MAE, RMSE, R²")

PRODUCTION RUNTIME PREDICTOR

Model: GradientBoosting
Features: Top 50

Hyperparameters:
  n_estimators: 794
  max_depth: 5
  learning_rate: 0.011740
  min_samples_split: 3
  min_samples_leaf: 1
  subsample: 0.572111
  random_state: 42

CROSS-VALIDATED METRICS (5-fold GroupKFold)

  Fold 1: MAPE = 9.3%, MAE = 20623.99s, R² = 0.9882
  Fold 2: MAPE = 9.2%, MAE = 25989.48s, R² = 0.9734
  Fold 3: MAPE = 9.7%, MAE = 36050.26s, R² = 0.9420
  Fold 4: MAPE = 9.9%, MAE = 34826.43s, R² = 0.9714
  Fold 5: MAPE = 9.6%, MAE = 38884.61s, R² = 0.9663

  Overall MAPE:   9.6%
  Overall MedAPE: 6.9%
  Overall MAE:    31260.56s
  Overall RMSE:   121989.43s
  Overall R²:     0.9666

Production model trained on full dataset (1107 samples).

Model saved to models\runtime_model.pkl
  File size: 2994.3 KB

Verified model loads successfully.
  Model type: GradientBoostingRegressor
  Features: 50

SANITY CHECK ON TRAINING FILES

File                                          Prec    Thresh       True       Pred 